In [1]:
%pylab inline
import numpy as np
import scipy as sc
import pandas as pd

import seaborn as sns
#sns.set_style("whitegrid")
#sns.set_context("talk")
#rc('axes', labelsize=20, titlesize=20)

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import scipy.stats as ss

import timeit
import pickle

from ABC_PMC import ABC_sample, ABC_PMC

Populating the interactive namespace from numpy and matplotlib


In [2]:
######
# set up for the normal ABC example
######

prior_mean = -4.0
prior_sd = 3
likelihood_sd = 1
original_data_size = 100

def NormalPriorFunction(x):
    return ss.norm.pdf(x=x,loc=prior_mean, scale=prior_sd)

def NormalPriorSampler(n):
    return np.random.normal(loc=prior_mean, scale=prior_sd, size=n)

def NormalLiklihoodSimulator(n, param):
    #unknown mean
    return np.random.normal(loc=param, scale=likelihood_sd, size=n)
    
def NormalSummary(data):
    return np.mean(data, axis=0)

data = np.random.normal(loc=0,scale=likelihood_sd,size=original_data_size)

In [15]:
k = 2
requested_sample_size = np.linspace(start=50,stop=250,num=2)
tolerance_seq = np.linspace(start=1, stop= 0.01, num=4) #T = 4

In [16]:
ABC_benchmark = []

for sample_size in requested_sample_size:
    run_time_ABC = 0
    niter_ABC = 0
    times_for_iteration_ABC = 0
    for rep in range(k):
        #Run ABC
        start_time = timeit.default_timer()
        ABC_run = ABC_sample(NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, tolerance_seq[-1], data , sample_size)
        run_time_ABC += timeit.default_timer() - start_time
        niter_ABC += ABC_run[1]
        times_for_iteration_ABC += np.mean(ABC_run[2])
    run_time_ABC /= k
    niter_ABC /= k
    times_for_iteration_ABC /= k
    output_dict = {'sample_size': sample_size, 'run_time': run_time_ABC, 'niter':niter_ABC, 'times_1000_iter': times_for_iteration_ABC}
    ABC_benchmark.append(output_dict)

file_name = "data/ABC_benchmark_k_"+str(k)+"_tol_"+str(tolerance_seq[-1])+"_num_"+str(len(requested_sample_size))+".p"
pickle.dump(ABC_benchmark, open( file_name, "wb" ) ) #save result in a file
#You can load using:
#test = pickle.load( open( "data/ABC_benchmark_k_2_tol_001.p", "rb" ) )

print(pd.DataFrame(ABC_benchmark))



      niter  run_time  sample_size  times_1000_iter
0   27011.5  0.735801         50.0         0.027185
1  132518.5  3.974384        250.0         0.029984


In [18]:
PMC_benchmark = []

for sample_size in requested_sample_size:
    run_time_PMC = 0
    niter_PMC = 0
    times_for_iteration_PMC = 0
    for rep in range(k):
        #Run ABC
        start_time = timeit.default_timer()
        PMC_run = ABC_PMC(NormalPriorFunction,NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, tolerance_seq, data , sample_size)
        run_time_PMC += timeit.default_timer() - start_time
        niter_PMC += PMC_run[1]
        times_for_iteration_PMC += np.mean(PMC_run[2])
    run_time_PMC /= k
    niter_PMC /= k
    times_for_iteration_PMC /= k
    output_dict = {'sample_size': sample_size, 'run_time': run_time_PMC, 'niter':niter_PMC, 'times_1000_iter': times_for_iteration_PMC}
    PMC_benchmark.append(output_dict)
    
file_name = "data/PMC_benchmark_k_"+str(k)+"_tol_"+str(tolerance_seq[-1])+"_num_"+str(len(requested_sample_size))+"_T_"+str(len(tolerance_seq))+".p"
pickle.dump(PMC_benchmark, open( file_name, "wb" ) ) #save result in a file
#You can load using:
#test = pickle.load( open( "data/ABC_benchmark_k_2_tol_001.p", "rb" ) )

print(pd.DataFrame(PMC_benchmark))

/Users/Giulio/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


     niter  run_time  sample_size  times_1000_iter
0   3206.5  0.296737         50.0         0.075285
1  14966.0  1.804536        250.0         0.115797


In [19]:
#PMC benchmark for different values of T but fixed n
sample_size_seq = [100,250,500,1000]

PMC_benchmark2 = []
for sample_size in sample_size_seq:
    for T in range(2,11):
        tolerance_seq = np.linspace(start=1, stop= 0.01, num=T)
        run_time_PMC = 0
        niter_PMC = 0
        times_for_iteration_PMC = 0
        for rep in range(k):
            #Run ABC
            start_time = timeit.default_timer()
            PMC_run = ABC_PMC(NormalPriorFunction,NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, tolerance_seq, data , sample_size)
            run_time_PMC += timeit.default_timer() - start_time
            niter_PMC += PMC_run[1]
            times_for_iteration_PMC += np.mean(PMC_run[2])
        run_time_PMC /= k
        niter_PMC /= k
        times_for_iteration_PMC /= k
        output_dict = {'T':T, 'sample_size': sample_size, 'run_time': run_time_PMC, 'niter':niter_PMC, 'times_1000_iter': times_for_iteration_PMC}
        PMC_benchmark2.append(output_dict)

file_name = "data/PMC_benchmark2_k_"+str(k)+"_tol_"+str(tolerance_seq[-1])+"_num_"+str(len(sample_size_seq))+"_T_2_10.p"
pickle.dump(PMC_benchmark, open( file_name, "wb" ) ) #save result in a file
print(pd.DataFrame(PMC_benchmark2))

    T    niter  run_time  sample_size  times_1000_iter
0   2  12474.5  1.051216          100         0.086168
1   3   6893.5  0.608973          100         0.082191
2   4   6022.5  0.538576          100         0.082121
3   5   5689.0  0.527074          100         0.084157
4   6   4388.0  0.476946          100         0.098917
5   7   4866.0  0.501562          100         0.095021
6   8   5248.5  0.555326          100         0.100381
7   9   4953.0  0.553780          100         0.106747
8  10   4594.5  0.540318          100         0.114261
